In [1]:
# include("./DroneLibV2.jl")
include("./ReinforceDDPGDroneLib.jl")

run_model (generic function with 2 methods)

In [2]:
# Initialize environment
state_dim = 12 + 3 + 4
action_dim = 4
MAX_ACTION = 14000.0
MAX_ACTION = convert(Float64, MAX_ACTION)

# Initialize replay buffer
buffer = deserialize("training-buffer_1.dat")
# buffer = ReplayBuffer(state_dim, action_dim, 10000000)

# Initialize noise
noise = OUNoise(action_dim)

OUNoise([0.0, 0.0, 0.0, 0.0], 0.15, 0.0, 0.2)

In [3]:
# Initialize agent
# agent = DDPGAgent(state_dim, action_dim, MAX_ACTION)
agent = DDPGAgent(state_dim, action_dim, MAX_ACTION, "agent_critic_model_1.bson", "agent_actor_model_1.bson")

DDPGAgent(Actor(Chain(Dense(19 => 256, relu), BatchNorm(256), Dense(256 => 512, relu), BatchNorm(512), Dense(512 => 1024, relu), BatchNorm(1024), Dense(1024 => 1024, relu), BatchNorm(1024), Dense(1024 => 256, relu), BatchNorm(256), Dense(256 => 64, relu), BatchNorm(64), Dense(64 => 4, σ))), Actor(Chain(Dense(19 => 256, relu), BatchNorm(256), Dense(256 => 512, relu), BatchNorm(512), Dense(512 => 1024, relu), BatchNorm(1024), Dense(1024 => 1024, relu), BatchNorm(1024), Dense(1024 => 256, relu), BatchNorm(256), Dense(256 => 64, relu), BatchNorm(64), Dense(64 => 4, σ))), Critic(Chain(Dense(23 => 256), BatchNorm(256), Dense(256 => 1024), BatchNorm(1024), Dense(1024 => 256), BatchNorm(256), Dense(256 => 128), BatchNorm(128), Dense(128 => 1))), Critic(Chain(Dense(23 => 256), BatchNorm(256), Dense(256 => 1024), BatchNorm(1024), Dense(1024 => 256), BatchNorm(256), Dense(256 => 128), BatchNorm(128), Dense(128 => 1))), Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), Adam(0.002, (0.9, 0.999

In [4]:
# Train agent
rewards = train(agent, buffer, 256, 10000, 2.0, 1.0, 1e-3)

┌ Warning: Performing scalar indexing on task Task (runnable) @0x000002c843ff22f0.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArraysCore C:\Users\sowri\.julia\packages\GPUArraysCore\uOYfN\src\GPUArraysCore.jl:106


Episode: 1, Episodic Reward: -62294.551850336895. Total steps: 517
Episode: 2, Episodic Reward: -204056.00335209144. Total steps: 1049
Episode: 3, Episodic Reward: -22990.10787275621. Total steps: 368
Episode: 4, Episodic Reward: -19302.239250120645. Total steps: 289
Episode: 5, Episodic Reward: -32561.322313501794. Total steps: 475
Episode: 6, Episodic Reward: -48899.4974335141. Total steps: 479
Episode: 7, Episodic Reward: -68432.63095576169. Total steps: 694
Episode: 8, Episodic Reward: -39685.255396319706. Total steps: 361
Episode: 9, Episodic Reward: -20294.072988608474. Total steps: 280
Episode: 10, Episodic Reward: -90934.3930165105. Total steps: 917
Episode: 11, Episodic Reward: -25433.17562901804. Total steps: 280
Episode: 12, Episodic Reward: -115693.91442757202. Total steps: 942
Episode: 13, Episodic Reward: -36894.68901459803. Total steps: 596
Episode: 14, Episodic Reward: -19834.43462297728. Total steps: 282
Episode: 15, Episodic Reward: -60374.39140679018. Total steps: 96

In [ ]:
# sample!(buffer, 1)
# state = get_state(model_env.drone)
# println(state)
action = get_action(agent, model_env.action, model_env.target_position, get_state(model_env.drone))
println(action)
# println(model_env.target_position)
# println(vcat(state, action, model_env.target_position))

In [ ]:
eval_env = DroneEnv([1.0, 1.0, 1.0], 2.0)
evaluate(agent, eval_env, 10)

In [ ]:
# target_position = [rand(-10:10), rand(-10:10), rand(1:10)]
# target_position = convert(Array{Float64, 1}, target_position)
# println("Target position: $(target_position)")
# model_env = DroneEnv(target_position, 1.0)
# state_space = vcat(get_normalised_state(model_env.drone), model_env.target_position, model_env.action)
# output = agent.actor.model(state_space)
model_env = DroneEnv([1.0, 1.0, 1.0], 2.0)
state_spaces, motors_rpm = run_model(agent, model_env)
println(get_state(model_env.drone), model_env.action)
plotStateSpaces(state_spaces)
plotMotorRpms(motors_rpm)
# println(motors_rpm)

In [ ]:
# 2223.8812148571014
# 547.933466732502
# 681.3510209321976
# 1113.097071647644

state = get_normalised_state(model_env.drone)
addParamterNoise!(agent, 0.001)
action = get_action(agent, model_env.action, model_env.target_position, state)

In [ ]:
save_model(agent, "agent_actor_model.bson", "agent_critic_model.bson")

In [ ]:
generateNoise!(noise)

In [ ]:
### Update the agent weights with noise
# old_params = Flux.params(agent.actor.model)
# new_params = Flux.params(agent.actor.model)
# params_vector = convert(Array{Float64, 1}, Flux.params(agent.actor.model))
# for param in new_params
# 	# for i in 1:length(new_params)
# 	temp_state = zeros(size(param))
# 	# temp_state = zeros(size(new_params[i]))
# 	noise_rand = randn!(temp_state) * 0.1
# 	temp_state += noise_rand
# 	param = temp_state
# 	# noise_rand = randn!(ones(size(param))) * 0.1
# 	# noise_rand = noise_rand |> gpu
# 	# param .+= noise_rand
# end

println("Check if the parameters are changed: ", old_params != new_params)
# new_params = Flux.params(agent.actor.model)
# println("Check if the parameters are changed: ", old_params != new_params)

In [ ]:
println("Check if the parameters are changed: ", old_params != Flux.params(agent.actor.model))


In [ ]:
check_1 = copy(agent_params[1])
check_noise = randn!(ones(size(agent_params[1]))) * 0.1
check_noise = check_noise |> gpu
check_1 .+= check_noise
println("Check if the parameters are changed: ", check_1 != agent_params[1])

In [ ]:
generateNoise!(noise)

In [ ]:
position = env.drone.position
previous_position = env.previous_position
velocity = env.drone.velocity
orientation = env.drone.orientation
angular_velocity = env.drone.angular_velocity
target_position = env.target_position

# Calculate the distance to the target
dist_to_target = norm(position - target_position)

# Calculate the distance to the target in the previous step
prev_dist_to_target = norm(previous_position - target_position)

In [ ]:
using JLD2

In [ ]:
actor_model_state = Flux.state(agent.actor.model);
critic_model_state = Flux.state(agent.critic.model);

In [ ]:
jldsave("agent_actor_model.jld2"; actor_model_state)
jldsave("agent_critic_model.jld2"; critic_model_state)

In [ ]:
import Pkg; Pkg.add("BSON")

In [ ]:
actor_model_state = Flux.state(agent.actor.model);
critic_model_state = Flux.state(agent.critic.model);

using JLD2
jldsave("agent_actor_model.jld2"; actor_model_state)
jldsave("agent_critic_model.jld2"; critic_model_state)

In [ ]:
using BSON: @save, @load


In [ ]:
# temp_actor = Actor(state_dim, action_dim, max_action)
# temp_actor_model = Flux.cpu(temp_actor.model);
@load "agent_actor_model.bson" actor_model

In [ ]:
println(actor_model)

In [ ]:
model_state = JLD2.load("agent_actor_model.jld2", "actor_model_state");

In [ ]:
model = Actor(state_dim, action_dim, max_action)

In [ ]:
# model.model = model.model |> cpu
temp_model = model.model |> cpu
Flux.loadparams!(temp_model, Flux.params(model_state))

In [ ]:
Flux.loadmodel!(model.model, model_state)

In [ ]:
deserialize("file.dat")